# Vision AI Showcase - Exploration Notebook

This notebook demonstrates how to use the Vision AI Showcase framework for computer vision tasks.

## Contents
1. Setup and Configuration
2. Model Registry Overview
3. Loading and Preprocessing Data
4. Running Inference
5. Visualizing Results
6. Model Evaluation


## 1. Setup and Configuration

In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import torch

# Add project root to path
project_root = Path('../').resolve()
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

# Import project modules
from src.models.registry import ModelRegistry
from src.inference.pipeline import InferencePipeline
from src.data.loaders import ImageDataLoader
from src.utils.config import ConfigManager

print(f"Project root: {project_root}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
# Load configuration
config_manager = ConfigManager('../configs/default.yaml')
config = config_manager.get_config()

print("Configuration loaded:")
for key, value in config.items():
    print(f"  {key}: {value}")

## 2. Model Registry Overview

In [ ]:
# Initialize model registry
registry = ModelRegistry('../models')

# Create default models if none exist
if len(registry) == 0:
    print("No models found. Creating default models...")
    registry.create_default_models()

print(f"Number of available models: {len(registry)}")
print("\nAvailable models:")
for model_name in registry.get_available_models():
    info = registry.get_model_info(model_name)
    description = info['config'].get('description', 'No description')
    num_classes = info['config'].get('num_classes', 'Unknown')
    architecture = info['config'].get('architecture', 'Custom')
    print(f"  - {model_name}: {description}")
    print(f"    Architecture: {architecture}, Classes: {num_classes}")

## 3. Loading and Preprocessing Data

In [ ]:
# Load sample images
data_loader = ImageDataLoader('../data/samples')
image_info = data_loader.get_image_info()

print("Dataset information:")
print(f"  Total images: {image_info['num_images']}")
print(f"  File extensions: {image_info['extensions']}")
print(f"  Average size: {image_info['avg_width']}x{image_info['avg_height']}")

if image_info['sample_paths']:
    print(f"  Sample files: {image_info['sample_paths'][:3]}")

In [ ]:
# Visualize some sample images
if data_loader.image_paths:
    fig, axes = plt.subplots(2, 3, figsize=(12, 8))
    axes = axes.flatten()
    
    num_images = min(6, len(data_loader.image_paths))
    
    for i in range(num_images):
        image_path = data_loader.image_paths[i]
        image = data_loader.load_single_image(image_path)
        
        if image:
            axes[i].imshow(image)
            axes[i].set_title(f"{image_path.name}\n{image.size[0]}x{image.size[1]}")
            axes[i].axis('off')
        else:
            axes[i].text(0.5, 0.5, 'Failed to load', ha='center', va='center')
            axes[i].set_title(f"{image_path.name} (Error)")
    
    # Hide unused subplots
    for i in range(num_images, 6):
        axes[i].axis('off')
    
    plt.suptitle('Sample Images from Dataset')
    plt.tight_layout()
    plt.show()
else:
    print("No sample images found. Please run the sample image generation script first:")
    print("python ../scripts/generate_sample_images.py")

## 4. Running Inference

In [ ]:
# Load a model for inference
model_name = "imagenet_resnet50"

print(f"Loading model: {model_name}")
try:
    model = registry.load_model(model_name)
    print("Model loaded successfully!")
    
    # Display model information
    model_info = model.get_model_info()
    print(f"\nModel Information:")
    print(f"  Name: {model_info['name']}")
    print(f"  Total Parameters: {model_info['total_parameters']:,}")
    print(f"  Device: {model_info['device']}")
    print(f"  Input Size: {model_info['input_size']}")
    
except Exception as e:
    print(f"Error loading model: {e}")
    print(f"Available models: {registry.get_available_models()}")
    model = None

In [ ]:
# Create inference pipeline and run predictions
if model and data_loader.image_paths:
    pipeline = InferencePipeline(model)
    
    # Select first image for prediction
    test_image_path = data_loader.image_paths[0]
    print(f"Running inference on: {test_image_path.name}")
    
    # Load and display the image
    test_image = data_loader.load_single_image(test_image_path)
    
    plt.figure(figsize=(8, 6))
    plt.imshow(test_image)
    plt.title(f"Test Image: {test_image_path.name}")
    plt.axis('off')
    plt.show()
    
    # Run prediction
    try:
        result = pipeline.predict_single_pil(test_image)
        
        print(f"\nPrediction Results:")
        print(f"  Inference Time: {result.get('inference_time', 'N/A'):.4f} seconds")
        print(f"  Top Prediction: {result.get('top_class', 'Unknown')}")
        print(f"  Confidence: {result.get('top_confidence', 0):.4f}")
        
        # Display top predictions if available
        predictions = result.get('predictions', [])
        if predictions:
            print(f"\n  Top 5 Predictions:")
            for pred in predictions:
                print(f"    {pred['rank']}. {pred['class_name']}: {pred['percentage']:.2f}%")
        
    except Exception as e:
        print(f"Error during inference: {e}")
else:
    print("Skipping inference: model not loaded or no test images available")

## 5. Visualizing Results

In [ ]:
# Batch inference on multiple images
if model and len(data_loader.image_paths) > 1:
    print("Running batch inference...")
    
    # Limit to first 4 images for visualization
    test_paths = data_loader.image_paths[:4]
    results = pipeline.predict_batch([str(path) for path in test_paths])
    
    # Visualize results
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    for i, (image_path, result) in enumerate(zip(test_paths, results)):
        if i >= 4:
            break
            
        # Load and display image
        image = data_loader.load_single_image(image_path)
        axes[i].imshow(image)
        
        # Add prediction as title
        if 'top_class' in result:
            title = f"{image_path.name}\n{result['top_class']} ({result['top_confidence']:.2f})"
        else:
            title = f"{image_path.name}\n(Prediction failed)"
        
        axes[i].set_title(title, fontsize=10)
        axes[i].axis('off')
    
    plt.suptitle('Batch Inference Results', fontsize=14)
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    successful_predictions = sum(1 for r in results if 'top_class' in r)
    avg_inference_time = np.mean([r.get('inference_time', 0) for r in results if 'inference_time' in r])
    
    print(f"\nBatch Inference Summary:")
    print(f"  Successful predictions: {successful_predictions}/{len(results)}")
    print(f"  Average inference time: {avg_inference_time:.4f} seconds")
    print(f"  Throughput: {1/avg_inference_time:.2f} images/second")
else:
    print("Skipping batch inference visualization")

## 6. Model Evaluation

In [ ]:
# Performance benchmarking
if model and data_loader.image_paths:
    print("Running performance benchmark...")
    
    # Use subset of images for benchmarking
    benchmark_paths = [str(path) for path in data_loader.image_paths[:5]]  # Limit for demo
    
    try:
        benchmark_results = pipeline.benchmark_model(benchmark_paths, num_runs=2)
        
        print(f"\nBenchmark Results:")
        print(f"  Test Images: {benchmark_results['num_test_images']}")
        print(f"  Total Predictions: {benchmark_results['total_predictions']}")
        print(f"  Average Time: {benchmark_results['average_inference_time']:.4f}s")
        print(f"  Min Time: {benchmark_results['min_inference_time']:.4f}s")
        print(f"  Max Time: {benchmark_results['max_inference_time']:.4f}s")
        print(f"  Throughput: {benchmark_results['images_per_second']:.2f} images/second")
        
        # Visualize timing distribution
        times = [benchmark_results['min_inference_time'], 
                benchmark_results['average_inference_time'],
                benchmark_results['max_inference_time']]
        labels = ['Min', 'Average', 'Max']
        
        plt.figure(figsize=(8, 5))
        bars = plt.bar(labels, times, color=['green', 'blue', 'red'], alpha=0.7)
        plt.ylabel('Inference Time (seconds)')
        plt.title('Model Performance Benchmark')
        
        # Add value labels on bars
        for bar, time_val in zip(bars, times):
            plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001, 
                    f'{time_val:.3f}s', ha='center', va='bottom')
        
        plt.tight_layout()
        plt.show()
        
    except Exception as e:
        print(f"Benchmark failed: {e}")
else:
    print("Skipping benchmark: model not loaded or no test images")

## 7. Exploring Different Models

In [ ]:
# Compare different models on the same image
if data_loader.image_paths:
    test_image_path = data_loader.image_paths[0]
    test_image = data_loader.load_single_image(test_image_path)
    
    model_comparison = []
    
    # Test each available model
    for model_name in registry.get_available_models():
        print(f"Testing model: {model_name}")
        try:
            model = registry.load_model(model_name)
            pipeline = InferencePipeline(model)
            result = pipeline.predict_single_pil(test_image)
            
            model_comparison.append({
                'model_name': model_name,
                'prediction': result.get('top_class', 'N/A'),
                'confidence': result.get('top_confidence', 0),
                'inference_time': result.get('inference_time', 0),
                'success': True
            })
            
        except Exception as e:
            print(f"  Error: {e}")
            model_comparison.append({
                'model_name': model_name,
                'error': str(e),
                'success': False
            })
    
    # Display comparison results
    print(f"\nModel Comparison Results for {test_image_path.name}:")
    print("=" * 60)
    
    for comp in model_comparison:
        if comp['success']:
            print(f"Model: {comp['model_name']}")
            print(f"  Prediction: {comp['prediction']}")
            print(f"  Confidence: {comp['confidence']:.4f}")
            print(f"  Time: {comp['inference_time']:.4f}s")
        else:
            print(f"Model: {comp['model_name']} - FAILED")
            print(f"  Error: {comp['error']}")
        print()
else:
    print("No test images available for model comparison")

## 8. Summary and Next Steps

In [ ]:
print("Vision AI Showcase Exploration Complete!")
print("\n" + "="*50)
print("SUMMARY")
print("="*50)
print(f"✓ Loaded configuration from: {config_manager.config_path}")
print(f"✓ Available models: {len(registry)}")
print(f"✓ Sample images: {len(data_loader.image_paths)}")
print(f"✓ PyTorch device: {'CUDA' if torch.cuda.is_available() else 'CPU'}")
print("\nNext Steps:")
print("1. Try the web interface: streamlit run ../app.py")
print("2. Use the CLI: python ../main.py --mode demo")
print("3. Add your own images to ../data/samples/")
print("4. Create custom models by extending BaseModel")
print("5. Explore the scripts in ../scripts/")
print("\nFor more information, check the README.md file!")